In [1]:
from pydantic import BaseModel, Field, field_validator
from typing import Optional, Any, Dict, Callable, Union
from rich import print

JsonDict = Dict[str, Any]


def CliArg(default: Any = None, flag: Optional[str] = None, **kwargs) -> Any:
    """Wrapper around pydantic Field to add flag for cli commands."""
    extra: Union[JsonDict, Callable[[JsonDict], None]] = {"flag": flag} if flag else {}
    return Field(default=default, json_schema_extra=extra, **kwargs)


class CliCommand(BaseModel):
    """Base class for CLI commands to format arguments."""
    def to_cli_args(self) -> list[str]:
        args = []
        for field_name, field in self.model_fields.items():
            value = getattr(self, field_name)
            if value is None:
                continue

            flag = (
                field.json_schema_extra.get("flag") if field.json_schema_extra else None
            )

            if flag:
                if isinstance(value, bool):
                    if value:
                        args.append(flag)
                else:
                    args.extend([flag, str(value)])
            elif not isinstance(value, bool):
                args.append(str(value))

        return args

In [2]:
from pydantic import model_validator


class ProdigalCommand(CliCommand):
    
    mode: Optional[str] = CliArg(
        default="anon",
        flag="--mode",
        description="Specify mode (normal, train, or anon).",
    )
    input_file: str = CliArg(
        ...,
        flag="--input_file",
        description="Specify input file (default stdin)",
    )
    output_file: str = CliArg(
        flag="--output_file", description="Specify output file (default stdout)"
    )
    protein_output_file: Optional[str] = CliArg(
        default=None,
        flag="--protein_file",
        description="Specify protein translations file",
    )
    nucleotide_output_file: Optional[str] = CliArg(
        default=None,
        flag="--mrna_file",
        description="Specify nucleotide sequences file",
    )
    training_file: Optional[str] = CliArg(
        default=None, flag="--start_file", description="Specify complete starts file"
    )
    summary_file: Optional[str] = CliArg(
        default=None, flag="--summ_file", description="Specify summary statistics file"
    )
    output_format: Optional[str] = CliArg(
        default="gbk",
        flag="--output_format",
        description="Specify output format",
    )
    quiet: Optional[bool] = CliArg(
        default=None,
        flag="--quiet",
        description="Run quietly (suppress logging output)",
    )

    @field_validator("mode")
    def validate_mode(cls, v):
        if v not in ("normal", "train", "anon"):
            raise ValueError('mode must be "normal", "train", "anon"')
        return v

    @field_validator("output_format")
    def validate_output_format(cls, v):
        if v not in ["gbk", "gff", "sqn", "sco"]:
            raise ValueError(
                'output_format must be one of ["gbk", "gff", "sqn", "sco"]'
            )
        return v

    @model_validator(mode="after")
    def validate_output_file(cls, values):
        output_file = values.output_file
        output_format = values.output_format
        if output_format and output_file and not output_file.endswith(output_format):
            raise ValueError(
                f"Output file must end with the specified output format: .{output_format}"
            )
        return values

In [3]:
# Testing the implementation
p = ProdigalCommand(
    input_file="contigs.fna",
    mode="anon",
    output_file="output.gbk",
    protein_output_file="output.faa",
    nucleotide_output_file="output.ffn",
)
print(p)

print(p.to_cli_args())

ProdigalCommand(
    mode='anon',
    input_file='contigs.fna',
    output_file='output.gbk',
    protein_output_file='output.faa',
    nucleotide_output_file='output.ffn',
    training_file=None,
    summary_file=None,
    output_format='gbk',
    quiet=None
)

[
    '--mode',
    'anon',
    '--input_file',
    'contigs.fna',
    '--output_file',
    'output.gbk',
    '--protein_file',
    'output.faa',
    '--mrna_file',
    'output.ffn',
    '--output_format',
    'gbk'
]

In [4]:
# trying to initialize with invalid values
try:
    p = ProdigalCommand(input_file="contigs.fna", mode="doesn't exist", output_format="abc")
except ValueError as e:
    for error in e.errors():
        print(error)

{
    'type': 'value_error',
    'loc': ('mode',),
    'msg': 'Value error, mode must be "normal", "train", "anon"',
    'input': "doesn't exist",
    'ctx': {'error': ValueError('mode must be "normal", "train", "anon"')},
    'url': 'https://errors.pydantic.dev/2.7/v/value_error'
}

{
    'type': 'value_error',
    'loc': ('output_format',),
    'msg': 'Value error, output_format must be one of ["gbk", "gff", "sqn", "sco"]',
    'input': 'abc',
    'ctx': {'error': ValueError('output_format must be one of ["gbk", "gff", "sqn", "sco"]')},
    'url': 'https://errors.pydantic.dev/2.7/v/value_error'
}

In [5]:
# trying to initialise without input file
try:
    p = ProdigalCommand()
except ValueError as e:
    for error in e.errors():
        print(error)

{
    'type': 'missing',
    'loc': ('input_file',),
    'msg': 'Field required',
    'input': {},
    'url': 'https://errors.pydantic.dev/2.7/v/missing'
}

In [6]:
# trying to initialise with incompatible output file
try:
    p = ProdigalCommand(input_file="contigs.fna", output_format="gff", output_file="output.txt")
except ValueError as e:
    for error in e.errors():
        print(error)

{
    'type': 'value_error',
    'loc': (),
    'msg': 'Value error, Output file must end with the specified output format: .gff',
    'input': {'input_file': 'contigs.fna', 'output_format': 'gff', 'output_file': 'output.txt'},
    'ctx': {'error': ValueError('Output file must end with the specified output format: .gff')},
    'url': 'https://errors.pydantic.dev/2.7/v/value_error'
}